In [53]:
from HPFC_imports import *

# Global Variables
# The monthly forward means:
# The data here is random! Please use your own forwards
months_forward = [750,790,800,650,700,850,1000,991,841,884,771,820]
# DataBase start date: DataFrame's start date
start_date = '20111201'
# DataBase End date
end_date = '20210101'

# Last day of prediction horizon: Usually the last day of the year
last_day_prediction_horizon = '2022-12-31'

# Prediction year: the name you will use to save the excel files
prediction_year = '2022'
# Prediction Horizon:
start_prediction_horizon = '20220101'
end_prediction_horizon = '20230101'

# Holidays in prediction horizon
holidays = ['2022-1-1' ,'2022-4-23','2022-5-1' , '2022-5-2',
            '2022-5-3' , '2022-5-4','2022-5-19',
            '2022-7-8' , '2022-7-9' , '2022-7-10' , 
            '2022-7-11' , '2022-7-12' , '2022-7-15', '2022-8-30' 
            , '2022-10-28', '2022-10-29']

# Holiday coefficient
holiday_coeff = 0.7

# Read historic data
df = pd.read_excel('data.xlsx')
# Make an empty dataframe to preprocess the data
df_data = pd.DataFrame()
df_data['date'] = pd.date_range(start=start_date, end=end_date, freq='H')
df_data = df_data[:-1]

In [54]:
# Flatten the data so that hourly prices are given as rows
rows = []
for i in range(len(df)):
    column = df.iloc[i]
    rows.append(list(column))
rows = np.array(rows)
price_column = rows.reshape(-1,1)
df_data['price'] = price_column 
df_data.set_index('date' , inplace = True)
df = df_data
df_data.head()

,price
date,
2011-12-01 00:00:00,169.99
2011-12-01 01:00:00,150.01
2011-12-01 02:00:00,150.00
2011-12-01 03:00:00,149.99
2011-12-01 04:00:00,149.99


In [55]:
# Get monthly means from historic data
monthly_prices = df.resample('M').mean()
# Add date features to data so that we can merge on them:

monthly_prices['month'] = monthly_prices.index.month
monthly_prices['d-t'] = monthly_prices.index.dayofweek
monthly_prices['year'] = monthly_prices.index.year
monthly_prices['dayofYear'] = monthly_prices.index.dayofyear
monthly_prices['week'] = monthly_prices.index.isocalendar().week

In [56]:
# Add date features on historic data so that we can merge it with the monthly_prices DF
df['month'] = df.index.month
df['d-t'] = df.index.dayofweek
df['year'] = df.index.year
df['dayofYear'] = df.index.dayofyear
df['hour'] = df.index.hour
df['week'] = df.index.isocalendar().week
# Replace week 53 with 52 as most years dont have 53 weeks
df['week'] = df['week'].replace(53,52)

In [57]:
# Merge DataFrames
df = df.merge(monthly_prices[['price' , 'year' , 'month']] ,
              how = 'left' , on = ['year' , 'month'])

# Calculate deviation coefficient from monthly mean price
df['DeviationFromMonthlyCoefficient'] = df['price_x'] / df['price_y']

# Groupby month week daytype and hour. Similar days should have similar coefficients
coef = df.groupby(['month' , 'week' , 'd-t' , 'hour']).mean().reset_index()

In [58]:
df_months = pd.DataFrame()
df_months['month'] = pd.date_range(start = start_prediction_horizon , 
                                  end = end_prediction_horizon ,freq = 'M')
df_months['months_forward'] = list(months_forward)
df_months['month'] = df_months['month'].dt.month

In [59]:
# Create a dataframe to strore all hourly prediction for the prediction horizon
df_hours = pd.DataFrame()
# Add date features to merge 
df_hours['hours'] = pd.date_range(start = start_prediction_horizon,
                                 end = end_prediction_horizon, freq = 'H')
df_hours['month'] = df_hours['hours'].dt.month
df_hours['dayOfyear'] = df_hours['hours'].dt.dayofyear
df_hours['week'] = df_hours['hours'].dt.isocalendar().week
df_hours['hour'] = df_hours['hours'].dt.hour
df_hours['d-t'] = df_hours['hours'].dt.dayofweek

In [60]:
# Merge Coeeficients with with prediction horizon hours
df_hours = df_hours.merge(coef[['DeviationFromMonthlyCoefficient','month' , 'week' , 'd-t' , 'hour']] , how = 'left' 
                        , on = ['month' , 'week' , 'd-t' , 'hour'])

In [61]:
# Merge df_hours with forwards
df_hours = df_hours.merge(df_months , how = 'left' , on = 'month')


In [62]:
# Multiply deviation from monthly coefficient with monthly mean to get the price
df_hours['Price'] = df_hours['DeviationFromMonthlyCoefficient'] * df_hours['months_forward']

In [63]:
# Get daily prices
daily_prices = df_hours.groupby('dayOfyear').mean().reset_index()
daily_prices

,dayOfyear,month,week,hour,d-t,DeviationFromMonthlyCoefficient,months_forward,Price
0,1,1.0,52.0,11.04,5.04,0.973758,750.0,730.318375
1,2,1.0,52.0,11.50,6.00,0.734076,750.0,550.556633
2,3,1.0,1.0,11.50,0.00,0.887962,750.0,665.971544
3,4,1.0,1.0,11.50,1.00,0.893355,750.0,670.016324
4,5,1.0,1.0,11.50,2.00,0.955436,750.0,716.576857
...,...,...,...,...,...,...,...,...
360,361,12.0,52.0,11.50,1.00,1.019976,820.0,836.380369
361,362,12.0,52.0,11.50,2.00,1.018174,820.0,834.902553
362,363,12.0,52.0,11.50,3.00,0.989702,820.0,811.555616
363,364,12.0,52.0,11.50,4.00,0.977860,820.0,801.844888


In [64]:
# Create a dataframe to fill in daily prices for the prediction horizon
df_days = pd.DataFrame()
df_days['day'] = pd.date_range(start = start_prediction_horizon ,
                              end = end_prediction_horizon, freq = 'D')
df_days['month'] = df_days['day'].dt.month
df_days['dayOfyear'] = df_days['day'].dt.dayofyear
df_days['week'] = df_days['day'].dt.isocalendar().week
df_days = df_days.set_index('day')[:last_day_prediction_horizon]
df_days['Price'] = list(daily_prices['Price'])
df_days['d-t'] = df_days.index.dayofweek

In [65]:
df_hours.set_index('hours', inplace = True)
df_hours = df_hours[['Price']]
df_hours['dayofmonth'] = pd.to_datetime(df_hours.index).day

In [66]:
# Save daily prices as excel
df_days.to_excel(f'daily_prices{prediction_year}.xlsx')

In [67]:
# Holidays
df_hours['holiday'] = 0
for i in holidays:
    df_hours.loc[i, 'holiday'] = 1
df_hours[df_hours['holiday'] == 1]
df_hours['Price'] = np.where((df_hours.holiday == 1 ),
                             df_hours.Price * holiday_coeff ,df_hours.Price)

In [68]:
# In our approach the first 3-days of each month are overestimated. 
# We will replace it with the following weeks' data

df_hours['Previus168'] = df_hours['Price'].shift(-168)
df_hours['Price'] = np.where((df_hours.dayofmonth == 1) ,df_hours.Previus168 ,df_hours.Price)
df_hours['Price'] = np.where((df_hours.dayofmonth == 2) ,df_hours.Previus168 ,df_hours.Price)
df_hours['Price'] = np.where((df_hours.dayofmonth == 3) ,df_hours.Previus168 ,df_hours.Price)


In [70]:
#
df_hours[:-1].to_excel(f'hourly_predictions{prediction_year}.xlsx')

,Price,dayofmonth,holiday,Previus168
hours,,,,
2022-01-01 00:00:00,733.604523,1,1,733.604523
2022-01-01 01:00:00,708.281048,1,1,708.281048
2022-01-01 02:00:00,596.609091,1,1,596.609091
2022-01-01 03:00:00,502.584845,1,1,502.584845
2022-01-01 04:00:00,473.335642,1,1,473.335642
...,...,...,...,...
2022-12-31 19:00:00,852.946635,31,0,NaN
2022-12-31 20:00:00,836.379528,31,0,NaN
2022-12-31 21:00:00,796.164100,31,0,NaN


,Price,dayofmonth,holiday,Previus168
hours,,,,
2022-01-01 00:00:00,733.604523,1,1,733.604523
2022-01-01 01:00:00,708.281048,1,1,708.281048
2022-01-01 02:00:00,596.609091,1,1,596.609091
2022-01-01 03:00:00,502.584845,1,1,502.584845
2022-01-01 04:00:00,473.335642,1,1,473.335642
...,...,...,...,...
2022-12-31 20:00:00,836.379528,31,0,NaN
2022-12-31 21:00:00,796.164100,31,0,NaN
2022-12-31 22:00:00,772.644951,31,0,NaN
